In [1]:
import geopandas as gpd
import h3
from shapely.geometry import Polygon
import json

gdf_seoul = gpd.read_file("./metadata/seoul_simplified.json")
seoul_geometry = gdf_seoul.geometry.values[0]
geojson_boundary = json.loads(gpd.GeoSeries([seoul_geometry]).to_json())['features'][0]['geometry']

In [2]:
# geo to hex id
h3_hexes = h3.geo_to_cells(geojson_boundary, 7)

# hex to polygon
hex_features = []
for hex_id in h3_hexes:
    boundary = h3.cell_to_boundary(hex_id)
    boundary_lonlat = [(lon, lat) for lat, lon in boundary]
    polygon = Polygon(boundary_lonlat)
    hex_features.append({
        'type': 'Feature',
        'hex_id': hex_id,
        'geometry': polygon
    })

hex_gdf = gpd.GeoDataFrame(hex_features, crs='EPSG:4326')
hex_gdf.to_file('./metadata/seoul_h3_resolution7.geojson', driver='GeoJSON')

In [3]:
triangle_features = []

for idx, row in hex_gdf.iterrows():
    hex_poly = row['geometry']
    centroid = hex_poly.centroid  # 육각형 중심점 계산
    coords = list(hex_poly.exterior.coords)

    # 각 변을 중심점과 연결하여 삼각형 생성
    for i in range(len(coords) - 1):
        triangle = Polygon([coords[i], coords[i + 1], (centroid.x, centroid.y)])
        triangle_features.append({
            'hex_id': row['hex_id'],
            'geometry': triangle
        })

triangle_gdf = gpd.GeoDataFrame(triangle_features, crs='EPSG:4326')

In [4]:
triangle_gdf.to_file('./metadata/seoul_h3_triangles_res7.geojson', driver='GeoJSON')